In [1]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats

import statsmodels.api as sm
from statsmodels.formula.api import ols

from statsmodels.stats.anova import AnovaRM

from helper import *

# Motivated Behaviors

## 1. Manual Analysis

We measured three behaviors ("Approach hands to food (n)," "Bring the food to the mouth (n)," and "Drop the food (n)") and "Food intake (g)" in the manual analysis. One can use the sample in `Examples/Palatable_Manual.csv` to reproduce the same results as below. First, read the csv file as follows:

In [16]:
df = pd.read_csv('Examples/Palatable_Manual.csv', index_col=0, header=[0, 1])
df

A            B            C        
                                 CNO Vehicle  CNO Vehicle  CNO Vehicle
Approach hands to food (n)        30      21   37      27   80      33
Bring the food to the mouth (n)  588     340  405     236  303     217
Drop the food (n)                  3      53   20      33   15      53
Food intake (g)                  214      76  162     140  622     266

Comparing CNO-injected and control subjects for each index by the paired t-test, we have:

In [17]:
print('{:>35}{:>25}'.format('Index', 'p-value (abs. value)'))

for idx in df.index:
    data = df.loc[idx].values.reshape(3, 2)
    p = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
    print('{:>35}{:>25.3f}'.format(idx, p))

                              Index     p-value (abs. value)
         Approach hands to food (n)                    0.221
    Bring the food to the mouth (n)                    0.070
                  Drop the food (n)                    0.091
                    Food intake (g)                    0.221


We also divided the values of CNO-injected subjects by the corresponding values of control subjects, and compared the rates of changes with 0, so that we can observe the rate of change caused by CNO injection.

In [18]:
print('{:>35}{:>25}'.format('Index', 'p-value (rate of change)'))

for idx in df.index:
    data = df.loc[idx].values.reshape(3, 2)
    p = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1]-1, popmean=0).pvalue
    print('{:>35}{:>25.3f}'.format(idx, p))

                              Index p-value (rate of change)
         Approach hands to food (n)                    0.163
    Bring the food to the mouth (n)                    0.030
                  Drop the food (n)                    0.050
                    Food intake (g)                    0.155


## 2. Deep Learning-based analysis

We measured three behaviors ("Tray Approach (n)," "Bout (n)," and "Drop the food (n)") and "Duration in Food Zone (s)" in the deep learning-based analysis. One can use the samples in `Examples/Palatable_DLC_MonkeyX_Y.csv (X = A / B / C, Y = CNO / Vehicle)` to reproduce the same results as below (those are raw outputs from DeepLabCut). We computed some parameters for each video, such as `Examples/Palatable_DLC_params.csv`.

* `fz x` is the x-coordinate of the food zone.
* `trayx` and `trayy` are the x- and y-coordinate of the tray, respectively.
* `tray-hand dist` is the threshold distance between the tray and hand, which is a criterion for "Tray Approach."
* `tray-mouth dist` is the threshold distance between the tray and mouth, which is a criterion for "Bout."

In [5]:
params_df = pd.read_csv('Examples/Palatable_DLC_params.csv', index_col=[0, 1], header=0)
params_df

fz x  trayx  trayy  tray-hand dist  tray-mouth dist
A CNO       847   1137    459       97.693398        22.203603
  Vehicle   843   1142    455      106.404887        20.248457
B CNO       809   1133    450      120.904921        21.189620
  Vehicle   866   1147    454       94.540996        21.587033
C CNO       906   1168    457       94.366308        22.472205
  Vehicle   845   1139    455       89.157165        17.492856

Based on those parameters, we computed each index for each video. Refer to `helper.py` for the detailed description of each function.

In [14]:
cases = [(i, j) for i in ['A', 'B', 'C'] for j in ['CNO', 'Vehicle']]
filenames = ['Examples/Palatable_DLC_Monkey{}_{}.csv'.format(i, j) for (i, j) in cases]

values = np.array([])

for case, filepath in zip(cases, filenames):
    # Approach
    params = tuple(params_df.loc[case, ['trayx', 'trayy', 'tray-hand dist']])
    coords, _, _ = return_approach(filepath, params, ll_crit=0.9, absolute=True, interval=0.2, FPS=24.0)
    appr = coords['Approach'].sum()

    # Bout
    params = tuple(params_df.loc[case, ['trayx', 'trayy', 'tray-mouth dist', 'tray-hand dist']])
    coords, _, _ = return_bout1(filepath, params, latency1=3.0, latency2=1.0, ll_crit=0.9, interval=0.2, FPS=24.0)
    bout = coords['Bout'].sum()

    # In Food Zone
    fz_x = params_df.loc[case, 'fz x']
    coords, _, _ = return_infz(filepath, fz_x, ll_crit=0.9, absolute=True)
    infz = coords['In'].sum() / 24


    values = np.concatenate((values, np.array([appr, bout, infz])))

values = values.reshape(3, 2, 3)

values

array([[[ 55.        ,  31.        , 866.83333333],
        [ 50.        ,  20.        , 217.58333333]],

       [[121.        ,  74.        , 828.54166667],
        [ 80.        ,  44.        , 204.        ]],

       [[268.        , 112.        , 520.45833333],
        [ 66.        ,  18.        , 309.375     ]]])

Then we compared (1) CNO-injected and control subjects by paired t-test and (2) the rates of changes with 0 by one-sample t-test, as above.

In [15]:
indices = ["Tray Approach (n)", "Bout (n)", "Duration in Food Zone (s)"]

print('{:>30}{:>25}{:>25}'.format('Index', 'p-value (abs. value)', 'p-value (rate of change)'))

for i, idx in enumerate(indices):
    data = values[:, :, i]
    p1 = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
    p2 = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1] - 1, popmean=0).pvalue
    print('{:>30}{:>25.3f}{:>25.3f}'.format(idx, p1, p2))

                         Index     p-value (abs. value) p-value (rate of change)
             Tray Approach (n)                    0.306                    0.317
                      Bout (n)                    0.215                    0.296
     Duration in Food Zone (s)                    0.073                    0.103


## 3. Food Motivation During Immobility Sessions


Again, one can use the samples in `Examples/Palatable_DLC_MonkeyX_Y.csv (X = A / B / C, Y = CNO / Vehicle)` and `Examples/Palatable_DLC_params.csv` to reproduce the same results as below.

### 3.1. Average Speed Inside Food Zone

We computed the average speed of the body center of the subject while it is located in the food zone. Then we compared (1) CNO-injected and control subjects by paired t-test and (2) the rates of changes with zero by one-sample t-test, as above.

In [31]:
params_df = pd.read_csv('Examples/Palatable_DLC_params.csv', index_col=[0, 1], header=0)

cases = [(i, j) for i in ['A', 'B', 'C'] for j in ['CNO', 'Vehicle']]
filenames = ['Examples/Palatable_DLC_Monkey{}_{}.csv'.format(i, j) for (i, j) in cases]

values = []

for case, filepath in zip(cases, filenames):

    fz_x = params_df.loc[case, 'fz x']
    coords, _, _ = return_infz(filepath, fz_x, ll_crit=0.9, absolute=True)
    bcx = coords[('Body_center', 'x')].values
    bc = coords[('Body_center')].values
    dx = bc[1:] - bc[:-1]
    v = dx[(bcx[1:] > fz_x) & (bcx[:-1] > fz_x)]
    vbar = np.linalg.norm(v, axis=1).sum() * 24 / len(v)


    values.append(vbar)

values = np.array(values).reshape(3, 2)

data = values

p1 = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
p2 = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1]-1, popmean=0).pvalue

pd.DataFrame(index=['Average Speed Inside Food Zone (px/s)'], data=[[p1, p2]], columns=['p-value (abs. value)', 'p-value (rate of change)'])

,p-value (abs. value),p-value (rate of change)
Average Speed Inside Food Zone (px/s),0.119913,0.006361


### 3.2. Total Duration of Immobility Sessions

In [32]:
crit = 0.25
filter = 360
latency = 36
dur = 36

lowspeed_len = []

for (case, filepath) in zip(cases, filenames):
    df = pd.read_csv(filepath, index_col=0, header=[1, 2], skiprows=0)
    lowspeed_sessions = get_lowspeed(df, crit, filter, latency, dur)
    lowspeed_len.append((lowspeed_sessions[:, 1] - lowspeed_sessions[:, 0]).sum())

lowspeed_len = np.array(lowspeed_len)
data = lowspeed_len.reshape(-1, 2) / 24

p1 = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
p2 = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1]-1, popmean=0).pvalue

pd.DataFrame(index=['Total Duration of Immobility Sessions (s)'], data=[[p1, p2]], columns=['p-value (abs. value)', 'p-value (rate of change)'])


,p-value (abs. value),p-value (rate of change)
Total Duration of Immobility Sessions (s),0.078078,0.023594


### 3.3. Behaviors in Immobility Sessions

The data for "Approach hands to food" (AH) are provided in `Examples/Palatable_Log_Example.csv`.

- Number of AH in immobility sessions (IS)
- Duration of IS containing AH

In [39]:
log = pd.read_csv('Examples/Palatable_Log_Example.csv', index_col=0)

num_in = []
num_out = []
lowspeed_len = []
bhvs_len = []

bhv = 'Approach hands to food (from tray)'

for (case, filepath) in zip(cases, filenames):
    df = pd.read_csv(filepath, index_col=0, header=[1, 2], skiprows=0)
    manual = (log.loc[(log[['Cycle', 'Subject', 'CNO / Vehicle', 'Behavior']] == ('3cycle', *case, bhv)).all(axis=1), 'Time_Relative_sf'].values * 24.0).astype(int)

    in_count, out_count = get_foc_behaviors_count(df, manual, crit, filter, latency, dur)
    num_in.append(in_count)
    num_out.append(out_count)

    lowspeed_sessions = get_lowspeed(df, crit, filter, latency, dur)
    lowspeed_len.append((lowspeed_sessions[:, 1] - lowspeed_sessions[:, 0]).sum())

    lowspeed_bhv_sessions = get_foc_sessions(df, manual, crit, filter, latency, dur)
    bhvs_len.append((lowspeed_bhv_sessions[:, 1] - lowspeed_bhv_sessions[:, 0]).sum())

num_in = np.array(num_in)
num_out = np.array(num_out)
lowspeed_len = np.array(lowspeed_len)
bhvs_len = np.array(bhvs_len)

ps = []

data = num_in.reshape(-1, 2)

p1 = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
p2 = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1]-1, popmean=0).pvalue

ps.append(p1)
ps.append(p2)


data = bhvs_len.reshape(-1, 2)

p1 = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
p2 = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1]-1, popmean=0).pvalue

ps.append(p1)
ps.append(p2)

ps = np.array(ps)

pd.DataFrame(index=['Number of AH in IS', 'Duration of IS containing AH'],
            columns=['p-value (abs. value)', 'p-value (rate of change)'],
            data=ps.reshape(2, 2)
)

,p-value (abs. value),p-value (rate of change)
Number of AH in IS,0.038041,0.014570
Duration of IS containing AH,0.098537,0.100704


- Proportion of IS containing AH to total IS

In [40]:
data = (bhvs_len / lowspeed_len).reshape(-1, 2)

p1 = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue

pd.DataFrame(index=['Proportion of IS containing AH to total IS'], data=[[p1]], columns=['p-value'])

,p-value
Proportion of IS containing AH to total IS,0.01673


# Abnormal Behaviors and Vocalization

## 4. Abnormal Behaviors

We measured the number of abnormal behaviors in each experimental condition. One can use the sample in `Examples/Abnormals.csv` to reproduce the same results as below.
Here, Condition 1, 2, 3, and 4 corresponds to the experimental conditions with toy, palatable food, water, and unpalatable food, respectively.

In [27]:
df = pd.read_csv('Examples/Abnormals.csv', header=[0, 1], index_col=0)
df

A           B        
          CNO Vehicle CNO Vehicle
Condition                        
1           1      50   7      85
2           0      62   0      59
3           1      40  33      59
4          11      12   5      60

For each subject, we conducted two-way ANOVA to determine whether the effect of the number of total abnormal behaviors caused by of CNO injection and the experimental condition is significant, respectively. The values at (`C(Cond)`, `PR(>F)`) and (`C(State)`, `PR(>F)`) are the p-values relevant to the significance of effect of the experimental condition and CNO injection, respectively.

In [50]:
# Monkey A
print('Monkey A\n')
results = df['A']

results = pd.DataFrame(
    data=results.values.reshape(-1), columns=['Number'], index=pd.MultiIndex.from_tuples( [(i, j) for i in range(1, 5) for j in ['CNO', 'Vehicle']])
)

results = results.reset_index()

results = results.rename(columns = {'level_0': 'Cond', 'level_1': 'State'})

model = ols('Number ~ C(Cond) + C(State)', data=results).fit()
table = sm.stats.anova_lm(model, typ=2)

print(table)

# Monkey B
print('\nMonkey B\n')

results = df['B']

results = pd.DataFrame(
    data=results.values.reshape(-1), columns=['Number'], index=pd.MultiIndex.from_tuples( [(i, j) for i in range(1, 5) for j in ['CNO', 'Vehicle']])
)

results = results.reset_index()

results = results.rename(columns = {'level_0': 'Cond', 'level_1': 'State'})

model = ols('Number ~ C(Cond) + C(State)', data=results).fit()
table = sm.stats.anova_lm(model, typ=2)

print(table)

Monkey A

            sum_sq   df         F    PR(>F)
C(Cond)    411.375  3.0  0.398089  0.765361
C(State)  2850.125  1.0  8.274223  0.063711
Residual  1033.375  3.0       NaN       NaN

Monkey B

          sum_sq   df          F    PR(>F)
C(Cond)    459.0  3.0   0.662816  0.628203
C(State)  5940.5  1.0  25.735018  0.014792
Residual   692.5  3.0        NaN       NaN


Next we normalized the values by dividing each value of a subject by the maximum numbers of abnormal behaviors attained by the very subject.

In [58]:
newdf = df.copy()
newdf['A'] = df['A'] / df['A'].values.max()
newdf['B'] = df['B'] / df['B'].values.max()

newdf

A                   B          
                CNO   Vehicle       CNO   Vehicle
Condition                                        
1          0.016129  0.806452  0.082353  1.000000
2          0.000000  1.000000  0.000000  0.694118
3          0.016129  0.645161  0.388235  0.694118
4          0.177419  0.193548  0.058824  0.705882

Then we conducted two-way repeated measures ANOVA to determine whether the effect of the number of total abnormal behaviors caused by of CNO injection, the experimental condition, and their interaction is significant, respectively. The values at (`Cond`, `PR > F`), (`State`, `PR > F`), and (`Cond:State`, `PR > F`) are the p-values relevant to the significance of effect of the experimental condition, CNO injection, and their interaction, respectively.

In [59]:
results = pd.DataFrame(
    data=newdf.values.reshape(-1), columns=['Normalized Value'], index=pd.MultiIndex.from_tuples( [(i, j, k) for i in range(1, 5) for j in ['A', 'B'] for k in ['CNO', 'Vehicle']])
)

results = results.reset_index()

results = results.rename(columns = {'level_0': 'Cond', 'level_1': 'Subject', 'level_2': 'State'})

aovrm2way = AnovaRM(results, 'Normalized Value', 'Subject', within=['Cond', 'State'])

print(aovrm2way.fit())

                  Anova
            F Value  Num DF Den DF Pr > F
-----------------------------------------
Cond          0.9736 3.0000 3.0000 0.5085
State      1497.2695 1.0000 1.0000 0.0164
Cond:State    1.3988 3.0000 3.0000 0.3947

